In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!unzip -q gdrive/My\ Drive/signate/test_1.zip
#!unzip -q gdrive/My\ Drive/signate/test_2.zip
#!unzip -q gdrive/My\ Drive/signate/test_3.zip
#!unzip -q gdrive/My\ Drive/signate/train_15.zip
!unzip -q gdrive/My\ Drive/signate/train_10.zip

#!ls

In [3]:
%matplotlib inline

import os
import cv2
import glob

import keras
import numpy as np
import matplotlib.pyplot as plt

import keras.backend as K
from keras.layers.convolutional import  Conv2D, MaxPooling2D
from keras.layers.core import Dense, Flatten,  Activation, Dropout
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from keras.initializers import he_normal
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

random_state = 42

#original loss function
def loss_function(y_true, y_pred):
    return -K.mean(29*y_true*K.log(y_pred + K.epsilon()) + (1-y_true)*K.log(1-y_pred + K.epsilon()), axis=-1)

Using TensorFlow backend.


In [0]:
class Dataset(object):
    def __init__(self):
        self.image_shape = (64, 64, 1)
        self.num_classes = 1
        
    def data_load(self):
        """
        データの読込
        """
        x = []
        y = []

        folder = ["nonTC/","TC/"]
        
        for idx, name in enumerate(folder):
            dpath = "train/"+name+"*.tif"
            files = glob.glob(dpath) #正規表現でファイル名を取得
            for _, file in enumerate(files):
                img = cv2.imread(file,-1)
                img = np.asarray(img)
                x.append(img)
                y.append(idx)
 
        x = np.array(x)
        y = np.array(y)
        return x, y
      
    def get_batch(self):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
        
        X_train, X_test = [self.preprocess(d) for d in [X_train, X_test]]
        y_train, y_test = [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
        
        return X_train, y_train, X_test, y_test
      
    def preprocess(self, data, label_data=False):
        if label_data:
            #convert class vectors to binary class matrices
            #data = keras.utils.to_categorical(data, self.num_classes)
           pass
        else:
            data = data.astype("float32")
         #   data /= 255 # convert the value to 0~1 scale
            shape = (data.shape[0], ) + self.image_shape #add dataset length
            data = data.reshape(shape)
                      
        return data 

In [0]:
class Trainer_aug(object):
    
    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(
            loss=loss, optimizer=optimizer, metrics=["accuracy"]
        )
        self.verbose = 1
        logdir = "log"
        self.log_dir = os.path.join(os.path.dirname('gdrive/My Drive/signate/'), logdir)
        self.model_file_name = "modef_file.hdf5"
  
        
    def train(self, X_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            shutil.rmtree(self.log_dir) #remove previous exectuion
        os.mkdir(self.log_dir)
        
        datagen= ImageDataGenerator(
            featurewise_center=False,
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=180,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=False,
            vertical_flip=False)
        
        # compute quantities for normalization (mean, std etc)
        datagen.fit(X_train)
        
        #split for validation data
        indices = np.arange(X_train.shape[0])
        np.random.shuffle(indices)
        validation_size = int(X_train.shape[0] * validation_split)
        X_train, X_valid = X_train[indices[:-validation_size], :], X_train[indices[-validation_size:], :]
        y_train,y_valid = y_train[indices[:-validation_size]], y_train[indices[-validation_size:]]

        #X_train, X_valid = X_train[indices[:-validation_size], :], X_train[indices[-validation_size:], :]
        #y_train,y_valid = y_train[indices[:-validation_size], :], y_train[indices[-validation_size:], :]

        
        model_path = os.path.join(self.log_dir, self.model_file_name)
        self._target.fit_generator(
           datagen.flow(X_train, y_train, batch_size=batch_size),
            steps_per_epoch=X_train.shape[0]//batch_size,
            epochs=epochs,
            validation_data=(X_valid, y_valid), 
            callbacks=[
                TensorBoard(log_dir=self.log_dir),
                ModelCheckpoint(model_path, save_best_only=True)
            ]
            , verbose=self.verbose
            , workers=4
        )

In [0]:
dataset = Dataset()

# train the model
x, y = dataset.data_load()
X_train, y_train, X_test, y_test = dataset.get_batch()

In [14]:
"""
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D

img_input = keras.layers.Input(shape=(64, 64, 1))
#model = MobileNet(input_tensor=img_input, alpha=0.5, weights=None, include_top=False)
base_model = MobileNet(input_tensor=img_input, alpha=0.5, weights=None, include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(128, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(1, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
"""

"\nfrom keras.applications import MobileNet\nfrom keras.layers import Dense, GlobalAveragePooling2D\n\nimg_input = keras.layers.Input(shape=(64, 64, 1))\n#model = MobileNet(input_tensor=img_input, alpha=0.5, weights=None, include_top=False)\nbase_model = MobileNet(input_tensor=img_input, alpha=0.5, weights=None, include_top=False)\n\n# add a global spatial average pooling layer\nx = base_model.output\nx = GlobalAveragePooling2D()(x)\n# let's add a fully-connected layer\nx = Dense(128, activation='relu')(x)\n# and a logistic layer -- let's say we have 200 classes\npredictions = Dense(1, activation='sigmoid')(x)\n\n# this is the model we will train\nmodel = Model(inputs=base_model.input, outputs=predictions)\n"

In [0]:
#load_model
from keras.models import model_from_json
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D,'loss_function': loss_function}):
    model = load_model('gdrive/My Drive/signate/test_sig_model.h5')

#print(model.summary())

In [16]:
trainer = Trainer_aug(model, loss=loss_function, optimizer="adam")
trainer.train(
    X_train, y_train, batch_size=64, epochs=5, validation_split=0.2
    )

Epoch 1/5
 207/1485 [===>..........................] - ETA: 59:56 - loss: 0.4948 - acc: 0.9034

KeyboardInterrupt: ignored

In [0]:
#--- save model weight
model.save('gdrive/My Drive/signate/test_sig_model.h5')

In [11]:
# show result
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

29701/29701 [==============================] - 18s 605us/step
Test loss: 0.4608179239999513
Test accuracy: 0.9053567219958923


In [12]:
from sklearn.metrics import classification_report
#from sklearn.metrics import precision_recall_fscore_support

y_evl = model.predict(X_test, batch_size=256)
y_evl2 = np.round(y_evl)
#y_evl2 = np.argmax(y_evl,axis=1)
#y_test2 = np.argmax(y_test,axis=1)
print(classification_report(y_test, y_evl2))

             precision    recall  f1-score   support

          0       1.00      0.91      0.95     28873
          1       0.21      0.88      0.34       828

avg / total       0.97      0.91      0.93     29701



In [13]:
y_evl = model.predict(X_train, batch_size=256)
y_evl2 = np.round(y_evl)
#y_evl2 = np.argmax(y_evl,axis=1)
#y_test2 = np.argmax(y_test,axis=1)
print(classification_report(y_train, y_evl2))

             precision    recall  f1-score   support

          0       1.00      0.91      0.95    115258
          1       0.23      0.89      0.36      3545

avg / total       0.97      0.91      0.93    118803



In [0]:
#-- submit用 --
import os
import glob
import cv2

import numpy as np
import pandas as pd

# input image dimensions
img_rows, img_cols = 64, 64

x_s = []
f_s = []

dpath = "test/*.tif"
files = glob.glob(dpath)

for i, file in enumerate(files):
  img = cv2.imread(file,-1)
  img = np.asarray(img)
  fnm = os.path.basename(file)  
  x_s.append(img)
  f_s.append(fnm)
  
x_s = np.asarray(x_s)

print(x_s.shape)

x_s = x_s.reshape(x_s.shape[0], img_rows, img_cols, 1)



(299135, 64, 64)


In [0]:
#load_model

from keras.models import model_from_json
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

with CustomObjectScope({'relu6': keras.applications.mobilenet.relu6,'DepthwiseConv2D': keras.applications.mobilenet.DepthwiseConv2D,'loss_function': loss_function}):
    model = load_model('gdrive/My Drive/signate/test_sig_model.h5')

#-- predict
batch_size = 128
y_s = model.predict(x_s, batch_size=batch_size)

#y_s2 = np.argmax(y_s,axis=1)
y_s2 = np.round(y_s)

#-- save submit data
df_y = pd.DataFrame(y_s2.reshape(-1,1)).astype(int)
df_f = pd.Series(f_s)

df_o = pd.concat([df_f, df_y],axis=1)
df_o.head()

df_o.to_csv("gdrive/My Drive/signate/submit.tsv", sep='\t', index=False, header=False)

In [8]:
df_o[:,1].astype(int)

TypeError: ignored